In [28]:
import pandas as pd
from database.market import Market
from transformer.date_transformer import DateTransformer
from datetime import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt

In [29]:
market = Market()
market.connect()
data = market.retrieve_data("btc")
market.close()

In [30]:
for col in data.columns:
    data.rename(columns={col:col.lower().replace(" ","")},inplace=True)
data = DateTransformer.convert_to_date(data,"date")

In [34]:
data

,_id,date,open,high,low,close,adjclose,volume
0,5fd76e9bfa80f804025a58ce,2014-09-17 00:00:00+00:00,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
1,5fd76e9bfa80f804025a58cf,2014-09-18 00:00:00+00:00,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2,5fd76e9bfa80f804025a58d0,2014-09-19 00:00:00+00:00,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
3,5fd76e9bfa80f804025a58d1,2014-09-20 00:00:00+00:00,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
4,5fd76e9bfa80f804025a58d2,2014-09-21 00:00:00+00:00,408.084991,412.425995,393.181000,398.821014,398.821014,26580100
...,...,...,...,...,...,...,...,...
2276,5fd76e9bfa80f804025a61b2,2020-12-10 00:00:00+00:00,18553.298828,18553.298828,17957.064453,18264.992188,18264.992188,25547132265
2277,5fd76e9bfa80f804025a61b3,2020-12-11 00:00:00+00:00,18263.929688,18268.453125,17619.533203,18058.904297,18058.904297,27919640985
2278,5fd76e9bfa80f804025a61b4,2020-12-12 00:00:00+00:00,18051.320313,18919.550781,18046.041016,18803.656250,18803.656250,21752580802
2279,5fd76e9bfa80f804025a61b5,2020-12-13 00:00:00+00:00,18806.765625,19381.535156,18734.332031,19142.382813,19142.382813,25450468637


In [106]:
holdings = {"btc":{},"queue":0,"bid":0}
trades = []
for exit in tqdm(range(0,20,5)):
    for floor in tqdm([True, False]):
        for signal in [1,-1]:
            for window in range(5,40,5):
                btc = data.copy()
                btc["rolling"] = btc["adjclose"].rolling(window=int(window)).mean()
                btc.dropna(inplace=True)
                for delta_requirement in range(0,25,5):
                    for date in btc["date"]:
                        offering = btc[btc["date"]==date]
                        offering_price = offering["adjclose"].item()
                        low = offering["low"].item()
                        high = offering["high"].item()
                        rolling = offering["rolling"]
                        if holdings["btc"] != {}:
                            if (offering_price - holdings["btc"]["buy_price"]) / holdings["btc"]["buy_price"] > delta_requirement/100 or \
                             (exit != 0 and (date- holdings["btc"]["buy_date"]).days >= exit):
                                trade = holdings["btc"]
                                trade["sell_price"] = offering_price
                                trade["sell_date"] = date
                                trade["dr"] = delta_requirement
                                trade["window"] = window
                                trade["signal"] = signal
                                trade["floor"] = floor
                                trade["exit"] = exit
                                trades.append(trade)
                                holdings["btc"] = {}
                        if holdings["queue"] == 1 and holdings["bid"] >= low and holdings["bid"] <= high:
                            holdings["btc"] = {"buy_date":date,"buy_price":holdings["bid"]}
                            holdings["queue"] = 0 
                            holdings["bid"] = 0
                        offering_delta = (offering["rolling"].item() - offering_price) / offering_price 
                        if holdings["btc"] == {}:
                            if floor:
                                if (signal > 0 and offering_delta > 0) or (signal < 0 and offering_delta < 0):
                                    holdings["queue"] = 1
                                    holdings["bid"] = offering_price
                            else:
                                if (signal > 0 and offering_delta > delta_requirement/100) or (signal < 0 and offering_delta < -delta_requirement/100):
                                    holdings["queue"] = 1
                                    holdings["bid"] = offering_price

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [13:58<00:00, 209.61s/it]


In [107]:
ta = pd.DataFrame(trades)
analysis = []
ta["gains"] = (ta["sell_price"] - ta["buy_price"]) / ta["buy_price"]
ta["hp"] = (ta["sell_date"] - ta["buy_date"]) 
ta["hp"] = [x.days for x in ta["hp"]]
for exit in tqdm(range(0,80,10)):
    for floor in [True,False]:
        for signal in [1,-1]:
            for window in tqdm(range(10,105,5)):
                for dr in range(0,105,5):
                    dt = ta[(ta["dr"] == dr) & (ta["window"] == window) 
                            & (ta["signal"] == signal)
                             & (ta["floor"] == floor)
                            & (ta["exit"] == exit)
                           ]
                    initial = 100
                    for gains in dt["gains"]:
                        initial = initial * (1 + gains)
                    analysis.append({
                        "signal":signal,
                        "dr":dr
                        ,"window":window
                        ,"cash":initial
                        ,"floor":floor
                        ,"exit":exit
                        ,"hp":dt["hp"].mean()
                    ,"trades":len(dt)})

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 31.46it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 31.47it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 31.32it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 31.26it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 31.25it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 31.65it/s]

100%|█████████████████████████████

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 30.81it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 29.02it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 30.30it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 31.08it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 31.33it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 31.18it/s]

100%|█████████████████████████████

In [108]:
a = pd.DataFrame(analysis)

In [111]:
a.sort_values("cash",ascending=False).head(20)

,signal,dr,window,cash,floor,exit,hp,trades
2080,-1,5,30,5454.176848,True,10,6.353488,215
2059,-1,5,25,5425.818614,True,10,6.260274,219
2794,-1,5,10,4129.959943,False,10,5.439189,148
2815,-1,5,15,3822.945867,False,10,5.659864,147
2836,-1,5,20,3232.758077,False,10,5.689189,148
2038,-1,5,20,2948.305293,True,10,6.327434,226
2878,-1,5,30,2407.797605,False,10,6.026667,150
2857,-1,5,25,2320.049397,False,10,5.901961,153
2856,-1,0,25,2241.565296,False,10,3.328767,365
2835,-1,0,20,2084.196169,False,10,3.401070,374
